In [1]:
import pandas as pd
df = pd.read_csv("secondary_data.csv", delimiter=';')

In [2]:
for x in df.columns:
    print(x, len(df[df[x].isna()]))

class 0
cap-diameter 0
cap-shape 0
cap-surface 14120
cap-color 0
does-bruise-or-bleed 0
gill-attachment 9884
gill-spacing 25063
gill-color 0
stem-height 0
stem-width 0
stem-root 51538
stem-surface 38124
stem-color 0
veil-type 57892
veil-color 53656
has-ring 0
ring-type 2471
spore-print-color 54715
habitat 0
season 0


In [3]:
# stem-root, stem-surface, veil-type, veil-color, spore-print-color

df = df.drop(columns=['stem-root', 'stem-surface',
                      'veil-type', 'veil-color',
                      'spore-print-color'], axis=1)

df = df.dropna()

In [4]:
df

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
2118,p,6.76,b,t,w,f,e,c,w,12.59,12.37,w,t,l,d,u
2119,p,6.83,b,t,w,f,e,c,w,13.84,13.54,w,t,l,d,u
2120,p,9.10,b,t,w,f,e,c,w,13.79,13.68,w,t,e,d,a
2121,p,8.27,b,t,w,f,e,c,w,12.63,12.46,w,t,l,d,a
2122,p,7.95,b,t,w,f,e,c,w,13.10,13.78,w,t,e,d,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61064,p,1.18,s,s,y,f,f,f,f,3.93,6.22,y,f,f,d,a
61065,p,1.27,f,s,y,f,f,f,f,3.18,5.43,y,f,f,d,a
61066,p,1.27,s,s,y,f,f,f,f,3.86,6.37,y,f,f,d,u
61067,p,1.24,f,s,y,f,f,f,f,3.56,5.44,y,f,f,d,u


In [9]:
[x for x in df.columns]

['class',
 'cap-diameter',
 'cap-shape',
 'cap-surface',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-attachment',
 'gill-spacing',
 'gill-color',
 'stem-height',
 'stem-width',
 'stem-color',
 'has-ring',
 'ring-type',
 'habitat',
 'season']